In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import os, sys, zipfile
import urllib.request
import shutil
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [2]:
print("os: %s" % os.name)
print("sys: %s" % sys.version)
print("numpy: %s, %s" % (np.__version__, np.__file__))


os: nt
sys: 3.7.5 (default, Oct 31 2019, 15:18:51) [MSC v.1916 64 bit (AMD64)]
numpy: 1.17.4, C:\Users\UX490\Anaconda3\envs\images\lib\site-packages\numpy\__init__.py


In [3]:
dataType='train2017'
annFile=r'C:\Users\UX490\Desktop\coco\annotations/instances_{}.json'.format(dataType)

In [4]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=28.19s)
creating index...
index created!


In [5]:
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
furniture indoor accessory animal electronic person vehicle outdoor food appliance sports kitchen


In [6]:
hotdogCats = coco.getCatIds(catNms=["hot dog"]);
hotdogIds = coco.getImgIds(catIds=hotdogCats);


In [7]:
len(set(hotdogIds))

1222

In [8]:
allcats = [cat['name'] for cat in cats]

In [9]:
allIds = []

for category in allcats:
    Cats = coco.getCatIds(catNms=category)
    Ids = coco.getImgIds(catIds=Cats)
    for ids in Ids:
        allIds.append(ids)  

In [10]:
len(set(allIds))

116657

In [11]:
NohotdogIds = set(allIds)-set(hotdogIds)

In [12]:
len(list(NohotdogIds))

115553

In [13]:
115553 + 1222

116775

In [14]:
allIds= set(allIds)

In [15]:
weird = []
for num in hotdogIds:
    if num not in allIds:
        weird.append(num)

In [17]:
imgIds = coco.getImgIds(imgIds = weird)
img = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
I = io.imread(img['coco_url'])
plt.axis('off')
plt.imshow(I)
plt.show()

C:\Users\UX490\Anaconda3\envs\images\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  


In [18]:
normal = set(hotdogIds) - set(weird)

In [19]:
len(NohotdogIds)

115553

In [20]:
import random

no_hotdog = random.sample(NohotdogIds, 1222)

In [21]:
len(no_hotdog)

1222

In [22]:
len(hotdogIds)

1222

In [23]:
hotdog = hotdogIds

In [24]:
len(hotdog)

1222

In [25]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from PIL import Image

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

Using TensorFlow backend.


In [58]:
imgIds = coco.getImgIds(imgIds = hotdog)
img = coco.loadImgs(imgIds[2])[0]
I = io.imread(img['coco_url'])
I.shape

(480, 640, 3)

In [28]:
x = I.reshape((1,) + I.shape)  

In [29]:
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir=r'C:\Users\UX490\Desktop\coco/preview', save_prefix='hotdog', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [ ]:
# there are two issues. I need a train and validation set and secondly not 
# all pictures have the same size, looking at the pictures 480,640,3 
# seems like a good shape as many pictures have that.

In [42]:
# 1 creating the train dataset. 

train_hotdog = random.sample(hotdog, 1000)
train_no_hotdog = random.sample(no_hotdog, 1000)

In [50]:
# 2 creating the validation dataset

val_hotdog = list(set(hotdog) - set(train_hotdog))
val_no_hotdog = list(set(no_hotdog) - set(train_no_hotdog))

In [87]:
import PIL
import os


In [92]:
# I create the directories for train and val while resizing pictures if necessary

# 1. First for train and hotdog
new_width  = 640
new_height = 480
name = "hotdog_"
imgIds = coco.getImgIds(imgIds = train_hotdog)
num =0
for i in range(0,len(train_hotdog)):
    img = coco.loadImgs(imgIds[i])[0]
    I = io.imread(img['coco_url'])
    pic=PIL.Image.fromarray(I, mode=None)
    pic = pic.resize((new_width, new_height))
    pic.save(os.path.join(r"C:\Users\UX490\Desktop\coco\Train\hotdog", name + str(num)) + '.JPG', 'JPEG', quality=90)
    num+=1

In [93]:
# 2. Train and no_hotdog

name = "no_hotdog_"
imgIds = coco.getImgIds(imgIds = train_no_hotdog)
num =0
for i in range(0,len(train_no_hotdog)):
    img = coco.loadImgs(imgIds[i])[0]
    I = io.imread(img['coco_url'])
    pic=PIL.Image.fromarray(I, mode=None)
    pic = pic.resize((new_width, new_height))
    pic.save(os.path.join(r"C:\Users\UX490\Desktop\coco\Train\no_hotdog", name + str(num)) + '.JPG', 'JPEG', quality=90)
    num+=1  

In [94]:
# 3. Val and hotdog

name = "val_hotdog_"
imgIds = coco.getImgIds(imgIds = val_hotdog)
num =0
for i in range(0,len(val_hotdog)):
    img = coco.loadImgs(imgIds[i])[0]
    I = io.imread(img['coco_url'])
    pic=PIL.Image.fromarray(I, mode=None)
    pic = pic.resize((new_width, new_height))
    pic.save(os.path.join(r"C:\Users\UX490\Desktop\coco\Val\hotdog", name + str(num)) + '.JPG', 'JPEG', quality=90)
    num+=1  

In [95]:
# 4. Val and no_hotdog

name = "val_no_hotdog_"
imgIds = coco.getImgIds(imgIds = val_no_hotdog)
num =0
for i in range(0,len(val_no_hotdog)):
    img = coco.loadImgs(imgIds[i])[0]
    I = io.imread(img['coco_url'])
    pic=PIL.Image.fromarray(I, mode=None)
    pic = pic.resize((new_width, new_height))
    pic.save(os.path.join(r"C:\Users\UX490\Desktop\coco\Val\no_hotdog", name + str(num)) + '.JPG', 'JPEG', quality=90)
    num+=1  

In [101]:
# so everything is ready to go. time to train the model

from tensorflow.python.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense



# dimensions of our images.
img_width, img_height = 640, 480

train_data_dir = r"C:\Users\UX490\Desktop\coco/Train"
validation_data_dir = r"C:\Users\UX490\Desktop\coco/Val"
nb_train_samples = 2000
nb_validation_samples = 444
epochs = 15
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))


model.add(Conv2D(32, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))


model.add(Conv2D(64, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))


model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Found 2000 images belonging to 2 classes.
Found 444 images belonging to 2 classes.
Epoch 1/15
125/125 [==============================] - 1279s 10s/step - loss: 0.7826 - accuracy: 0.6040 - val_loss: 0.6527 - val_accuracy: 0.6528
Epoch 2/15
125/125 [==============================] - 1903s 15s/step - loss: 0.6528 - accuracy: 0.6760 - val_loss: 0.5940 - val_accuracy: 0.7083
Epoch 3/15
125/125 [==============================] - 1396s 11s/step - loss: 0.5874 - accuracy: 0.7390 - val_loss: 0.5316 - val_accuracy: 0.7616
Epoch 4/15
125/125 [==============================] - 1579s 13s/step - loss: 0.5696 - accuracy: 0.7375 - val_loss: 0.5136 - val_accuracy: 0.7778
Epoch 5/15
125/125 [==============================] - 2255s 18s/step - loss: 0.5583 - accuracy: 0.7425 - val_loss: 0.6526 - val_accuracy: 0.7269
Epoch 6/15
125/125 [==============================] - 1586s 13s/step - loss: 0.5406 - accuracy: 0.7620 - val_loss: 0.5508 - val_accuracy: 0.7199
Epoch 7/15
125/125 [===========================

In [1]:
# getting the 
epochs1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
val_accuracy1 = [0.6528, 0.7083, 0.7616, 0.7778, 0.7269, 0.7199, 0.7708, 0.7685, 0.7639, 0.7963, 0.7569, 0.7801, 0.7454, 0.7708, 0.7662]

In [2]:
len(epochs1) == len(val_accuracy1)

True

In [103]:
model.save("first_attempt.h5")

In [ ]:
from tensorflow.keras.models import load_model
model2 = load_model('first_try.h5')